## cell 1 (install):


In [ ]:
!pip install -r ../requirements.txt

: 

## Cell 2 (load env safely):

In [ ]:
from dotenv import load_dotenv
import os, getpass
load_dotenv()

# prefer .env; if not present, prompt
if not os.getenv("QDRANT_URL"):
    print("QDRANT_URL not found in .env. You can paste it now (input won't be saved):")
    os.environ["QDRANT_URL"] = getpass.getpass("QDRANT_URL: ")

if not os.getenv("QDRANT_API_KEY"):
    os.environ["QDRANT_API_KEY"] = getpass.getpass("QDRANT_API_KEY: ")

if not os.getenv("GROQ_API_KEY"):
    print("If you want to use Groq API (recommended for better generation) paste key now; or leave blank to use local fallback.")
    os.environ["GROQ_API_KEY"] = getpass.getpass("GROQ_API_KEY (optional): ")


## Cell 3 (index data):


In [ ]:
# run index (this will use Qdrant Cloud by default)
!python ../src/load_data.py --csv ../data/rag_optimized_5000.csv --max 1000
# remove --max to index all


##  Cell 4 (inference example):

In [ ]:
from src.rag import RAG
rag = RAG(use_qdrant=True)  # set False for local DB
query = "How do I reset my password?"
print("Query:", query)
print("Answer:\n")
print(rag.answer(query, top_k=5))


## Cell 5 (local fallback generation demo):

In [ ]:
rag_local = RAG(use_qdrant=False)  # local in-memory DB
# index a few docs quickly
from src.embeddings import Embedder
embedder = Embedder()
sample_docs = ["How to reset password: go to portal -> forgot password -> follow instructions.",
               "Library hours are 9am-9pm Monday-Friday, 10am-6pm on weekends."]
embs = embedder.embed(sample_docs)
from src.db_local import InMemoryDB
db = InMemoryDB()
db.upsert(sample_docs, embs)
rag_local.db = db

print(rag_local.answer("When is the library open?", top_k=2, use_groq_if_available=False))
